# $U(1)$ Gauge Model using L2HMC in graph mode

--------------------------------------------------------------------
### TODO:
* [ ] Look at performance on Cooley (longer training runs).
* [ ] Fit observables to Eq. \ref{eq:therm_time} to determine the thermalization time $\tau$.

\begin{equation} 
f(t) \equiv A \exp^{-t / \tau}+ \,\, B
\label{eq:therm_time}
\end{equation}
* [ ] Look at defining a distance metric as the difference in topological charge between two samples and see what effect adding this as an additional term to the loss function has on the models' ability to tunnel between topological sectors.

--------------------------------------------------------------------

In [1]:
import os
import sys
import time
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from scipy.special import i0, i1

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import utils.gauge_model_helpers as helpers
from utils.data_utils import (
    calc_avg_vals_errors, block_resampling, jackknife_err
)
from gauge_model import (
    GaugeModel, check_else_make_dir, save_params_to_pkl_file
)

tfe = tf.contrib.eager
%autoreload 2
%matplotlib notebook

#from gauge_model import GaugeModel
#%reload_ext gauge_model.GaugeModel
#from gauge_model import GaugeModel
#
#from dynamics.gauge_dynamics import GaugeDynamics
#%reload_ext dynamics.gauge_dynamics
#from dynamics.gauge_dynamics import GaugeDynamics

In [2]:
def autocorr(x):
    result = np.correlate(x, x, mode='full')
    result /= result[result.argmax()]
    return result[result.size//2:]

## Define parameters

In [3]:
params = {
#--------------------- Lattice parameters ----------------------------
    'time_size': 8,
    'space_size': 8,
    'link_type': 'U1',
    'dim': 2,
    'num_samples': 4,
    'rand': False,
#--------------------- Leapfrog parameters ---------------------------
    'num_steps': 5,
    'eps': 0.1,
    'loss_scale': 1.,
    'loss_eps': 1e-4,
#--------------------- Learning rate parameters ----------------------
    'learning_rate_init': 1e-3,
    'learning_rate_decay_steps': 500,
    'learning_rate_decay_rate': 0.96,
#--------------------- Annealing rate parameters ---------------------
    'annealing': True,
    'annealing_steps': 500,
    'annealing_factor': 0.99995,
    #'beta': 3.,
    'beta_init': 3.,
    'beta_final': 8.,
#--------------------- Training parameters ---------------------------
    'train_steps': 20000,
    'save_steps': 1000,
    'logging_steps': 50,
    'training_samples_steps': 1000,
    'training_samples_length': 100,
#--------------------- Model parameters ------------------------------
    'conv_net': True,
    'hmc': False,
    'eps_trainable': True,
    'metric': 'l2',
    'aux': False,
    'clip_grads': False,
    'clip_value': 10.,
}

## Create model

In [4]:
os.environ["KMP_BLOCKTIME"] = str(0)
os.environ["KMP_SETTINGS"] = str(1)
os.environ["KMP_AFFINITY"] = "granularity=fine,verbose,compact,1,0"
#os.environ["OMP_NUM_THREADS"] = str(2)

In [5]:
config = tf.ConfigProto()
tf.reset_default_graph()

In [6]:
model = GaugeModel(params=params, 
                   config=config,
                   sess=None,
                   log_dir=None,
                   restore=False)

save_params_to_pkl_file(params, model.info_dir)

Creating directory for new run: /Users/saforem2/ANL/l2hmc/gauge_logs_graph/run_48/
Creating directory: /Users/saforem2/ANL/l2hmc/gauge_logs_graph/run_48/samples_history.
Creating directory: /Users/saforem2/ANL/l2hmc/gauge_logs_graph/run_48/samples_history/training.
Creating directory: /Users/saforem2/ANL/l2hmc/gauge_logs_graph/run_48/train_samples.
time_size: 8
space_size: 8
link_type: U1
dim: 2
num_samples: 4
rand: False
num_steps: 5
eps: 0.1
loss_scale: 1.0
loss_eps: 0.0001
learning_rate_init: 0.001
learning_rate_decay_steps: 500
learning_rate_decay_rate: 0.96
annealing: True
annealing_steps: 500
annealing_factor: 0.99995
beta_init: 3.0
beta_final: 8.0
train_steps: 20000
save_steps: 1000
logging_steps: 50
training_samples_steps: 1000
training_samples_length: 100
conv_net: True
hmc: False
eps_trainable: True
metric: l2
aux: False
clip_grads: False
clip_value: 10.0
--------------------------------------------------------------------------------

Building graph... (started at: Sat Feb  

In [7]:
model.train_steps = 10000
model.annealing_factor = 0.9999

In [8]:
model.train(model.train_steps, kill_sess=False)

----------------------------------------------------------------------------------------------------
     STEP           LOSS       NORM. TIME     ACCEPT %        EPS           BETA           LR      
----------------------------------------------------------------------------------------------------
     0/20000       -1436         246.4         0.617          0.1            3           0.001    
     5/20000       7193         0.2935       2.02e-06       0.09618        3.002         0.001    
    10/20000       7437         0.2566        0.06769       0.0937         3.003         0.001    
    15/20000       7257         0.2607         0.25         0.09158        3.005         0.001    
    20/20000       -77.9        0.3027        0.2543        0.08904        3.006         0.001    
    25/20000      -340.2        0.2752        0.6424        0.08695        3.008         0.001    
    30/20000      -418.7        0.2597        0.7974        0.08558        3.009         0.001    
    3

   400/20000      -73.49        0.2765        0.6377        0.07393        3.122         0.001    
   405/20000      -134.7        0.3261        0.8733        0.07382        3.124         0.001    
   410/20000      -212.8        0.2681        0.9285        0.07369        3.126         0.001    
   415/20000       -192         0.2801        0.9842        0.07355        3.127         0.001    
   420/20000      -183.7        0.2913        0.8979        0.0734         3.129         0.001    
   425/20000      -160.3        0.2774        0.8291        0.07326        3.13          0.001    
   430/20000      -179.2        0.2932        0.9079        0.0731         3.132         0.001    
   435/20000      -102.8        0.3113        0.8719        0.07298        3.133         0.001    
   440/20000      -148.2        0.2763        0.9651        0.0729         3.135         0.001    
   445/20000       -175          0.272        0.9403        0.07281        3.137         0.001    
   450/200

   815/20000      -180.7        0.2653        0.9481        0.06083        3.255        0.00096   
   820/20000      -164.2         0.283        0.8826        0.06062        3.256        0.00096   
   825/20000      -181.2        0.2692        0.9529        0.0604         3.258        0.00096   
   830/20000      -147.7        0.2637         0.832        0.06019        3.26         0.00096   
   835/20000      -174.2        0.2682         0.914        0.05996        3.261        0.00096   
   840/20000      -160.1         0.288        0.9469        0.05972        3.263        0.00096   
   845/20000       -152         0.2666        0.9539        0.05949        3.265        0.00096   
   850/20000      -174.3        0.2598        0.9363        0.05927        3.266        0.00096   
   855/20000      -164.6        0.2624        0.9664        0.05908        3.268        0.00096   
   860/20000      -167.3        0.2639        0.8607        0.05892        3.269        0.00096   
   865/200

  1100/20000      -110.9        0.2792        0.9653        0.04976        3.349       0.0009216  
  1105/20000      -151.8        0.2873        0.9484        0.04959        3.351       0.0009216  
  1110/20000      -143.2        0.2856           1          0.04943        3.352       0.0009216  
  1115/20000      -151.3        0.2681           1          0.04929        3.354       0.0009216  
  1120/20000      -143.9        0.2786        0.9685        0.04914        3.356       0.0009216  
  1125/20000      -134.5        0.2817        0.9658        0.04899        3.357       0.0009216  
  1130/20000      -224.9        0.2637        0.9382        0.04882        3.359       0.0009216  
  1135/20000      -153.8        0.2818        0.9485        0.04861        3.361       0.0009216  
  1140/20000      -160.2        0.2799        0.9323        0.0484         3.362       0.0009216  
  1145/20000      -139.4        0.2831        0.9888        0.04818        3.364       0.0009216  
  1150/200

  1515/20000      -49.83         0.293        0.9937        0.03673        3.491       0.0008847  
  1520/20000      -48.88        0.2612        0.9879        0.0366         3.493       0.0008847  
  1525/20000      -106.7        0.2737        0.9292        0.03652        3.494       0.0008847  
  1530/20000       -96.6         0.253           1          0.03644        3.496       0.0008847  
  1535/20000      -93.37        0.2532        0.8908        0.03637        3.498       0.0008847  
  1540/20000      -160.5         0.272        0.9522        0.03622        3.499       0.0008847  
  1545/20000      -140.9        0.2716        0.9269        0.03599        3.501       0.0008847  
  1550/20000       -151         0.2675           1          0.03578        3.503       0.0008847  
  1555/20000      -136.6        0.2773        0.9661        0.03557        3.505       0.0008847  
  1560/20000      -100.6        0.2681        0.9643        0.03539        3.507       0.0008847  
  1565/200

...

  8535/20000      -28.57        0.2754        0.9666        0.01274        7.044       0.0004996  
  8540/20000      -18.62        0.2546        0.9667        0.01254        7.047       0.0004996  
  8545/20000      -26.72        0.2613        0.9787        0.01265        7.051       0.0004996  
  8550/20000      -67.44        0.2784        0.9904        0.01315        7.054       0.0004996  
  8555/20000      -55.59         0.255        0.9828        0.01356        7.058       0.0004996  
  8560/20000      -65.24        0.2436        0.9815        0.01298        7.061       0.0004996  
  8565/20000      -47.56        0.2578        0.9612        0.01254        7.065       0.0004996  
  8570/20000      -36.43        0.2501        0.9608        0.0124         7.069       0.0004996  
  8575/20000      -66.65        0.2347        0.9801        0.01256        7.072       0.0004996  
  8580/20000      -66.77        0.2234        0.9947        0.01249        7.076       0.0004996  
  8585/200

  8950/20000      -17.87        0.2686        0.9666        0.01211        7.342       0.0004996  
  8955/20000      -57.57        0.2625        0.9948        0.01205        7.346       0.0004996  
  8960/20000      -57.47        0.2299         0.984        0.01212        7.35        0.0004996  
  8965/20000      -8.198        0.2395        0.9596        0.01214        7.353       0.0004996  
  8970/20000      -36.81        0.2656         0.974        0.01295        7.357       0.0004996  
  8975/20000      -38.67        0.2845        0.9941        0.01402        7.361       0.0004996  
  8980/20000       -56.8        0.2635        0.9863        0.01435        7.364       0.0004996  
  8985/20000      -57.39        0.2675        0.9711        0.0136         7.368       0.0004996  
  8990/20000      -28.08        0.2646        0.9705        0.01326        7.372       0.0004996  
  8995/20000      -28.59        0.2708        0.9968        0.0131         7.375       0.0004996  
Saving che

  9235/20000      -26.49        0.2527        0.9661        0.01369        7.555       0.0004796  
  9240/20000      -37.58         0.283        0.9647        0.01239        7.558       0.0004796  
  9245/20000      -18.16        0.2487        0.9713        0.01181        7.562       0.0004796  
  9250/20000      -56.12        0.2797        0.9807        0.01202        7.566       0.0004796  
  9255/20000       -65.8        0.2642        0.9731        0.01252        7.57        0.0004796  
  9260/20000      -84.94        0.2506        0.9612        0.01271        7.574       0.0004796  
  9265/20000      -38.45        0.2501        0.9785        0.0128         7.577       0.0004796  
  9270/20000      -48.12        0.2707        0.9711        0.01303        7.581       0.0004796  
  9275/20000        -48         0.2449        0.9757        0.01318        7.585       0.0004796  
  9280/20000        -56         0.2892        0.9571        0.01363        7.589       0.0004796  
  9285/200

  9650/20000      -27.47        0.2701        0.9935        0.01239        7.875       0.0004604  
  9655/20000      -38.11        0.2661        0.9615        0.0125         7.879       0.0004604  
  9660/20000        -29         0.2635        0.9974        0.01287        7.883       0.0004604  
  9665/20000      -28.54        0.2568        0.9869        0.01335        7.887       0.0004604  
  9670/20000      -36.39        0.2382         0.968        0.01399        7.891       0.0004604  
  9675/20000      -36.86        0.2537        0.9818        0.01389        7.894       0.0004604  
  9680/20000      -28.23        0.2681        0.9938        0.01364        7.898       0.0004604  
  9685/20000      -47.31        0.2569        0.9695        0.01396        7.902       0.0004604  
  9690/20000      -48.71        0.2677        0.9913        0.0141         7.906       0.0004604  
  9695/20000      -56.22        0.2581        0.9756        0.01424        7.91        0.0004604  
  9700/200

0

In [9]:
run_steps = [100, 200, 500, 1000, 5000, 10000]

In [10]:
run_steps1 = [101, 201, 501, 1001]#, 5001]

In [11]:
for steps in run_steps:
    _ = model.run(steps)

Running (trained) L2HMC sampler for 100 steps...
step:      0/100    accept prob (avg):  0.9713   step size: 0.01361 	 time/step: 0.1674

accept prob:  [0.947116   0.97005177 0.96811944 1.        ]


step:     10/100    accept prob (avg):  0.9552   step size: 0.01361 	 time/step: 0.1424

accept prob:  [1.         0.97465175 0.962066   0.88390416]


step:     20/100    accept prob (avg):   0.961   step size: 0.01361 	 time/step: 0.1357

accept prob:  [0.9083824  1.         0.93577373 1.        ]


step:     30/100    accept prob (avg):   0.97    step size: 0.01361 	 time/step: 0.09497

accept prob:  [0.9826167 1.        0.8973254 1.       ]


step:     40/100    accept prob (avg):  0.9288   step size: 0.01361 	 time/step: 0.0763

accept prob:  [0.9686333  1.         0.8248233  0.92165685]


step:     50/100    accept prob (avg):  0.9419   step size: 0.01361 	 time/step: 0.09632

accept prob:  [0.9148355 0.9278392 1.        0.9247701]


step:     60/100    accept prob (avg):  0.9421   st

step:    200/500    accept prob (avg):  0.9676   step size: 0.01361 	 time/step: 0.08328

accept prob:  [1.         0.909335   0.9754183  0.98577535]


step:    210/500    accept prob (avg):  0.9572   step size: 0.01361 	 time/step: 0.08004

accept prob:  [0.903826  1.        0.9248375 1.       ]


step:    220/500    accept prob (avg):     1     step size: 0.01361 	 time/step: 0.08096

accept prob:  [1. 1. 1. 1.]


step:    230/500    accept prob (avg):  0.9782   step size: 0.01361 	 time/step: 0.08735

accept prob:  [1.         0.914696   0.99793434 1.        ]


step:    240/500    accept prob (avg):  0.9763   step size: 0.01361 	 time/step: 0.1009

accept prob:  [1.         0.9430334  1.         0.96207917]


step:    250/500    accept prob (avg):     1     step size: 0.01361 	 time/step: 0.1498

accept prob:  [1. 1. 1. 1.]


step:    260/500    accept prob (avg):  0.9893   step size: 0.01361 	 time/step: 0.2261

accept prob:  [1.        1.        0.9573167 1.       ]


step:    27

step:    230/1000   accept prob (avg):  0.9092   step size: 0.01361 	 time/step: 0.1035

accept prob:  [0.9186083  0.870392   1.         0.84783125]


step:    240/1000   accept prob (avg):  0.9951   step size: 0.01361 	 time/step: 0.1357

accept prob:  [0.9803791 1.        1.        1.       ]


step:    250/1000   accept prob (avg):  0.9881   step size: 0.01361 	 time/step: 0.1256

accept prob:  [0.9569896 0.9952718 1.        1.       ]


step:    260/1000   accept prob (avg):  0.9956   step size: 0.01361 	 time/step: 0.1205

accept prob:  [1.       0.982282 1.       1.      ]


step:    270/1000   accept prob (avg):  0.9768   step size: 0.01361 	 time/step: 0.1557

accept prob:  [1.        1.        0.9692247 0.9378312]


step:    280/1000   accept prob (avg):     1     step size: 0.01361 	 time/step: 0.1449

accept prob:  [1. 1. 1. 1.]


step:    290/1000   accept prob (avg):  0.9659   step size: 0.01361 	 time/step: 0.1533

accept prob:  [1.        0.9394572 1.        0.9239981]



step:    790/1000   accept prob (avg):  0.9832   step size: 0.01361 	 time/step: 0.08602

accept prob:  [0.9459394  1.         1.         0.98666084]


step:    800/1000   accept prob (avg):   0.985   step size: 0.01361 	 time/step: 0.08105

accept prob:  [1.        0.9781558 0.9618546 1.       ]


step:    810/1000   accept prob (avg):  0.9817   step size: 0.01361 	 time/step: 0.09666

accept prob:  [0.94463074 1.         0.98234457 1.        ]


step:    820/1000   accept prob (avg):   0.93    step size: 0.01361 	 time/step: 0.1063

accept prob:  [0.7868065  0.96767586 1.         0.9654796 ]


step:    830/1000   accept prob (avg):  0.9716   step size: 0.01361 	 time/step: 0.1115

accept prob:  [1.         0.88640535 1.         1.        ]


step:    840/1000   accept prob (avg):  0.9717   step size: 0.01361 	 time/step: 0.1254

accept prob:  [1.        1.        1.        0.8866078]


step:    850/1000   accept prob (avg):  0.9234   step size: 0.01361 	 time/step: 0.1058

accept pro

step:    320/5000   accept prob (avg):  0.9854   step size: 0.01361 	 time/step: 0.09073

accept prob:  [1.         1.         0.94167376 1.        ]


step:    330/5000   accept prob (avg):  0.9221   step size: 0.01361 	 time/step: 0.09281

accept prob:  [0.87529916 0.9981642  0.96008396 0.8549633 ]


step:    340/5000   accept prob (avg):  0.9463   step size: 0.01361 	 time/step: 0.1106

accept prob:  [0.89778405 0.96709967 0.9816849  0.93859047]


step:    350/5000   accept prob (avg):  0.9259   step size: 0.01361 	 time/step: 0.1023

accept prob:  [0.90386635 1.         0.92745847 0.8720893 ]


step:    360/5000   accept prob (avg):  0.9561   step size: 0.01361 	 time/step: 0.09335

accept prob:  [1.         0.93123424 0.92808014 0.9649895 ]


step:    370/5000   accept prob (avg):  0.9949   step size: 0.01361 	 time/step: 0.09055

accept prob:  [1.         0.97973955 1.         1.        ]


step:    380/5000   accept prob (avg):  0.9603   step size: 0.01361 	 time/step: 0.09012



...

step:   7510/1e+04  accept prob (avg):  0.9626   step size: 0.01361 	 time/step: 0.1817

accept prob:  [0.96433246 1.         0.9399443  0.9461757 ]


step:   7520/1e+04  accept prob (avg):  0.9825   step size: 0.01361 	 time/step: 0.08375

accept prob:  [0.9299726 1.        1.        1.       ]


step:   7530/1e+04  accept prob (avg):     1     step size: 0.01361 	 time/step: 0.1049

accept prob:  [1. 1. 1. 1.]


step:   7540/1e+04  accept prob (avg):     1     step size: 0.01361 	 time/step: 0.07915

accept prob:  [1. 1. 1. 1.]


step:   7550/1e+04  accept prob (avg):  0.9282   step size: 0.01361 	 time/step: 0.09346

accept prob:  [1.         0.89926857 0.8427118  0.97071207]


step:   7560/1e+04  accept prob (avg):   0.96    step size: 0.01361 	 time/step: 0.1072

accept prob:  [1.         1.         0.83983624 1.        ]


step:   7570/1e+04  accept prob (avg):  0.9695   step size: 0.01361 	 time/step: 0.107 

accept prob:  [0.9248189 0.953018  1.        1.       ]


step:   7580

step:   8070/1e+04  accept prob (avg):  0.9558   step size: 0.01361 	 time/step: 0.1448

accept prob:  [0.99643326 0.9620899  1.         0.86448884]


step:   8080/1e+04  accept prob (avg):  0.9635   step size: 0.01361 	 time/step: 0.09415

accept prob:  [0.94051766 0.9133085  1.         1.        ]


step:   8090/1e+04  accept prob (avg):  0.9881   step size: 0.01361 	 time/step: 0.1288

accept prob:  [0.9789334  1.         0.97327304 1.        ]


step:   8100/1e+04  accept prob (avg):  0.9561   step size: 0.01361 	 time/step: 0.1201

accept prob:  [0.97278064 1.         0.85158926 1.        ]


step:   8110/1e+04  accept prob (avg):  0.9823   step size: 0.01361 	 time/step: 0.1578

accept prob:  [1.        1.        0.9290762 1.       ]


step:   8120/1e+04  accept prob (avg):  0.9712   step size: 0.01361 	 time/step: 0.1267

accept prob:  [1.         1.         0.88473046 1.        ]


step:   8130/1e+04  accept prob (avg):  0.9785   step size: 0.01361 	 time/step: 0.1184

accept p

step:   8630/1e+04  accept prob (avg):  0.9794   step size: 0.01361 	 time/step: 0.1238

accept prob:  [1.        1.        1.        0.9177896]


step:   8640/1e+04  accept prob (avg):     1     step size: 0.01361 	 time/step: 0.1335

accept prob:  [1. 1. 1. 1.]


step:   8650/1e+04  accept prob (avg):  0.9473   step size: 0.01361 	 time/step: 0.09158

accept prob:  [0.9283764 0.9559792 1.        0.9047368]


step:   8660/1e+04  accept prob (avg):  0.9761   step size: 0.01361 	 time/step: 0.1731

accept prob:  [1.        0.9042919 1.        1.       ]


step:   8670/1e+04  accept prob (avg):  0.9573   step size: 0.01361 	 time/step: 0.137 

accept prob:  [0.90291214 0.9400295  1.         0.98645437]


step:   8680/1e+04  accept prob (avg):  0.9782   step size: 0.01361 	 time/step: 0.1228

accept prob:  [0.9128523 1.        1.        1.       ]


step:   8690/1e+04  accept prob (avg):     1     step size: 0.01361 	 time/step: 0.1514

accept prob:  [1. 1. 1. 1.]


step:   8700/1e+04  ac

step:   9190/1e+04  accept prob (avg):  0.9968   step size: 0.01361 	 time/step: 0.08794

accept prob:  [1.         0.98711205 1.         1.        ]


step:   9200/1e+04  accept prob (avg):  0.9611   step size: 0.01361 	 time/step: 0.1255

accept prob:  [1.        0.9407467 0.9714109 0.9323005]


step:   9210/1e+04  accept prob (avg):  0.9742   step size: 0.01361 	 time/step: 0.1151

accept prob:  [0.9852609 1.        1.        0.9116454]


step:   9220/1e+04  accept prob (avg):  0.8811   step size: 0.01361 	 time/step: 0.1473

accept prob:  [0.8768601  0.872809   0.9200199  0.85489804]


step:   9230/1e+04  accept prob (avg):  0.9119   step size: 0.01361 	 time/step: 0.1803

accept prob:  [0.9300404 0.8831851 1.        0.8342663]


step:   9240/1e+04  accept prob (avg):  0.9548   step size: 0.01361 	 time/step: 0.2138

accept prob:  [0.9928647  1.         0.95049495 0.8757442 ]


step:   9250/1e+04  accept prob (avg):  0.9653   step size: 0.01361 	 time/step: 0.1294

accept prob:  [0

step:   9740/1e+04  accept prob (avg):  0.9838   step size: 0.01361 	 time/step: 0.121 

accept prob:  [1.        0.9353461 1.        1.       ]


step:   9750/1e+04  accept prob (avg):  0.9629   step size: 0.01361 	 time/step: 0.09465

accept prob:  [1.         1.         0.93567103 0.9158814 ]


step:   9760/1e+04  accept prob (avg):  0.9671   step size: 0.01361 	 time/step: 0.1189

accept prob:  [1.         0.9430226  1.         0.92546487]


step:   9770/1e+04  accept prob (avg):  0.9733   step size: 0.01361 	 time/step: 0.1256

accept prob:  [0.8933246 1.        1.        1.       ]


step:   9780/1e+04  accept prob (avg):     1     step size: 0.01361 	 time/step: 0.1236

accept prob:  [1. 1. 1. 1.]


step:   9790/1e+04  accept prob (avg):     1     step size: 0.01361 	 time/step: 0.1399

accept prob:  [1. 1. 1. 1.]


step:   9800/1e+04  accept prob (avg):  0.9613   step size: 0.01361 	 time/step: 0.122 

accept prob:  [0.9052279 1.        1.        0.9400567]


step:   9810/1e+04

In [39]:
import pickle

with open(model.files['parameters_pkl_file'], 'wb') as f:
    pickle.dump(model.params, f)

In [ ]:
model.sess.graph.collections

In [ ]:
model.sess.graph.get_collection

In [ ]:
model.dynamics.position_fn.summary()

In [ ]:
# Iterate over samples history and calculate observables for each sample.
# `lattice.calc_plaq_observables(samples)` calculates observables for each of
# the samples in the mini-batch.
actions_history = []
avg_plaquettes_history = []
top_charges_history = []
for idx, samples in enumerate(samples_history):
    t0 = time.time()
    observables = np.array(model.lattice.calc_plaq_observables(samples))
    actions, plaqs, charges = observables
    
    actions_history.append(actions)
    avg_plaquettes_history.append(plaqs)
    top_charges_history.append(charges)
    
    print(f'step: {idx}  '
          f'time / step: {time.time() - t0:^6.4g}  '
          f'avg action: {np.mean(actions):^6.4g}  '
          f'avg plaquette: {np.mean(plaqs):^6.4g} '
          f'top charge: {np.mean(charges):^6.4g}')

In [ ]:
_ = helpers.plot_run_data(model.data, 
                          model.params, 
                          model.steps_arr, 
                          model.figs_dir, 
                          skip_steps=1)

In [ ]:
#tf.reset_default_graph()
#model = GaugeModel(params=params,
#                   config=None,
#                   sess=None,
#                   conv_net=False,
#                   hmc=False,
#                   log_dir='../../gauge_logs_graph/run_25',
#                   restore=True)

In [ ]:
samples = np.random.randn(*model.samples.shape)
samples_history = []

In [ ]:
for i in range(500):
    t0 = time.time()
    samples = model.sess.run(model.x_out, feed_dict={model.x: samples})
    samples_history.append(samples)
    print(f'step: {i:^6.4g} time/step: {time.time() - t0:^6.4g}')

In [ ]:
samples_history_conv = np.array(samples_history_conv)
print(samples_history_conv.shape)

In [ ]:
import pickle
samples_history_file = os.path.join(model.info_dir, 'samples_history.pkl')
with open(samples_history_file, 'wb') as f:
    pickle.dump(samples_history_conv, f)